# Installing packages

In [ ]:
!pip install faiss-cpu
!pip install -U sentence-transformers
!pip install langchain-community
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4
   ━━━━━━

# Imports

In [ ]:
# "/content/drive/MyDrive/Colab Notebooks/GenAi/Data/icd10cm 1.pdf"

import os
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Creating embeddings**

In [ ]:
# Load the PDF document
pdf_path = "/content/drive/MyDrive/Colab Notebooks/GenAi/Data/icd10cm 1.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Split the documents into smaller chunks for embedding
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Create embeddings using SentenceTransformer
embedding_model = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
#Get the page content of each text chunk for embedding
page_contents = [text.page_content for text in texts]
embeddings = embedding_model.embed_documents(page_contents)

# Create text-embedding pairs using page contents instead of Document objects
text_embedding_pairs = zip(page_contents, embeddings)

# Build FAISS index
# !pip install faiss-cpu
from langchain_community.vectorstores import FAISS
faiss = FAISS.from_embeddings(text_embedding_pairs, embedding_model)

# Save the vector store for later use
# faiss.save()
# Save the FAISS index to a local directory
index_path = 'icd10cm_faiss_index' # Specify the directory to save the index
faiss.save_local(index_path)

# **Load embeddings**

In [ ]:
# Specify the path to the saved index
index_path = "/content/drive/MyDrive/Colab Notebooks/GenAi/icd10cm_faiss_index"

# Load the FAISS index
embedding_model = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
loaded_faiss = FAISS.load_local(index_path, embeddings=embedding_model, allow_dangerous_deserialization=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Prompt template

In [ ]:
# azure credentials
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

# dotenv.load_dotenv()

# Load the .env file
load_dotenv('/content/drive/MyDrive/Colab Notebooks/GenAi/.env')

deployment_name = os.environ['DEPLOYMENT_NAME']
api_key = os.environ["OPENAI_API_KEY"]
azure_endpoint = os.environ['AZURE_ENDPOINT']
api_version = os.environ['OPENAI_API_VERSION']

In [ ]:
chat = AzureChatOpenAI(temperature=0.0, model=deployment_name, azure_endpoint=azure_endpoint)
chat

AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7a06d53558a0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7a06d53efbb0>, model_name='gpt-35-turbo', temperature=0.0, model_kwargs={}, openai_api_key='78d406f975874c20ac2298c6d83e2f7d', openai_proxy='', azure_endpoint='https://nitoropenai.openai.azure.com/', openai_api_version='2023-03-15-preview', openai_api_type='azure')

In [ ]:
template_string = """"Answer the question based on the context below.\
    If you don't know the answer, just say that you don't know, don't try to make up an answer.\
    Context: {contexts}.\
    Question: {query}
    Answer:
"""
query = "What is ICD code for Encounter for surgical aftercare following surgery on the sense organs?"

results = loaded_faiss.similarity_search_with_score(query, k=5)

contexts = [result[0].page_content for result in results]

# prompt_template
prompt_template = ChatPromptTemplate.from_template(template_string)

prompt = prompt_template.format_messages(
                    contexts=contexts,
                    query=query)

In [ ]:
prompt_template.messages[0].prompt.input_variables

['contexts', 'query']

In [ ]:
# Call the LLM to translate to the style of the customer message
response = chat(prompt)
print(response.content)

The ICD code for Encounter for surgical aftercare following surgery on the sense organs is Z48.810.


# ICD classification

In [ ]:
template1 = '''You are a medical expert. Based on the following patient symptoms and medical history,
classify the most appropriate ICD-10 code from the descriptions provided.

Patient Symptoms: {patient_description}
Please provide the most relevant ICD-10 code and a brief explanation for the classification.

Format the output as JSON with the following keys:
ICD code
explanation'''

description = '''The patient reports feeling unusually fatigued and thirsty over the past month, with increased urination, especially at night. The patient has also noticed unexplained weight loss despite eating regularly'''

prompt_template1 = ChatPromptTemplate.from_template(template1)

prompt = prompt_template1.format_messages(patient_description=description)


In [ ]:
ICD_Cls = chat(prompt)

print(ICD_Cls.content)

{
  "ICD code": "E10.65",
  "explanation": "The most relevant ICD-10 code for the patient's symptoms and medical history is E10.65. This code corresponds to Type 1 diabetes mellitus with hyperglycemia. The patient's symptoms of fatigue, increased thirst, increased urination, unexplained weight loss, and the presence of hyperglycemia are consistent with the diagnosis of Type 1 diabetes mellitus. Further diagnostic tests and evaluation may be required to confirm the diagnosis."
}


# Evaluation

In [ ]:
import pandas as pd

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/GenAi/Data/final_data.csv'

df = pd.read_csv(path)
df.head()

,Disease,ICD code,Description
0,Diabetes Mellitus Type 2,E11.9,The patient reports feeling unusually fatigued...
1,Asthma,J45.909,The patient reports frequent shortness of brea...
2,Migraine without Aura,G43.909,"The patient complains of severe, throbbing hea..."
3,Hypertension,I10,The patient reports occasional headaches and d...
4,Gastroesophageal Reflux Disease (GERD),K21.9,The patient describes a burning sensation in t...


In [ ]:
import json

def predict_icd_code(patient_description):
  prompt = prompt_template1.format_messages(patient_description=patient_description)
  ICD_Cls = chat(prompt)
  try:
    icd_code_json = json.loads(ICD_Cls.content)
    return icd_code_json.get('ICD code', None)
  except:
    return None


df['ICD Code Predicted'] = df['Description'].apply(predict_icd_code)

In [ ]:
df.head(10)

,Disease,ICD code,Description,ICD Code Predicted
0,Diabetes Mellitus Type 2,E11.9,The patient reports feeling unusually fatigued...,E11.9
1,Asthma,J45.909,The patient reports frequent shortness of brea...,J45.909
2,Migraine without Aura,G43.909,"The patient complains of severe, throbbing hea...",G43.1
3,Hypertension,I10,The patient reports occasional headaches and d...,I10
4,Gastroesophageal Reflux Disease (GERD),K21.9,The patient describes a burning sensation in t...,K21.9
5,Chronic Obstructive Pulmonary Disease (COPD),J44.9,The patient notes experiencing persistent shor...,J44.9
6,Depressive Disorder,F32.9,The patient indicates feeling persistently sad...,F32.9
7,Osteoarthritis of Knee,M17.9,The patient expresses chronic pain and stiffne...,M17.0
8,Rheumatoid Arthritis,M06.9,"The patient reports joint pain, swelling, and ...",M06.9
9,"Chronic Kidney Disease, Stage 3",N18.30,The patient states increased fatigue and swell...,N18.9


In [ ]:
# Exact prediction
(df['ICD code']== df['ICD Code Predicted']).sum()/df.shape[0]

0.5245901639344263

In [ ]:
# compare ICD code and ICD code predicted
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(df['ICD code'], df['ICD Code Predicted'])
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 52.46%


In [ ]:
# prompt: Compare first 3 elements of each value in column  'ICD code' and 'ICD Code Predicted'

def compare_first_three_elements(row):
  icd_code = row['ICD code']
  predicted_icd_code = row['ICD Code Predicted']

  if icd_code is not None and predicted_icd_code is not None:
    return icd_code[:3] == predicted_icd_code[:3]
  else:
    return False

df['First_Three_Match'] = df.apply(compare_first_three_elements, axis=1)

# Calculate the accuracy of the first three elements
accuracy_first_three = df['First_Three_Match'].sum() / len(df)
print(f"Accuracy of Considering only first three character: {accuracy_first_three:.2}")

Accuracy of Considering only first three character: 0.69


# **Chains**

## only for reference


template2 = '''You are a medical expert. Based on the following patient symptoms and medical history,
classify the most appropriate ICD-10 code from the descriptions provided.

Patient Symptoms: {description}
First classify the patient symptoms into following group: {ICD_code_group}
and then use this group to classify the most accurate ICD-10 code.
Please provide the most relevant ICD-10 code and a brief explanation for the classification.

Format the output as JSON with the following keys:
ICD code group
ICD code
explanation'''

ICD_code_group = '''
'A00–B99':'Certain infectious and parasitic diseases',
'C00–D48':'Neoplasms',
'D50–D89':'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism',
'E00–E90':'Endocrine, nutritional and metabolic diseases',
'F00–F99':'Mental and behavioural disorders',
'G00–G99':'Diseases of the nervous system',
'H00–H59':'Diseases of the eye and adnexa',
'H60–H95':'Diseases of the ear and mastoid process',
'I00–I99':'Diseases of the circulatory system',
'J00–J99':'Diseases of the respiratory system',
'K00–K93':'Diseases of the digestive system',
'L00–L99':'Diseases of the skin and subcutaneous tissue',
'M00–M99':'Diseases of the musculoskeletal system and connective tissue',
'N00–N99':'Diseases of the genitourinary system',
'O00–O99':'Pregnancy, childbirth and the puerperium',
'P00–P96':'Certain conditions originating in the perinatal period',
'Q00–Q99':'Congenital malformations, deformations and chromosomal abnormalities',
'R00–R99':'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified',
'S00–T98':'Injury, poisoning and certain other consequences of external causes',
'V01–Y98':'External causes of morbidity and mortality',
'Z00–Z99':'Factors influencing health status and contact with health services',
'U00–U99':'Codes for special purposes
'''

description = '''The patient reports feeling unusually fatigued and thirsty over the past month, with increased urination, especially at night. The patient has also noticed unexplained weight loss despite eating regularly'''

prompt_template2 = ChatPromptTemplate.from_template(template2)

prompt = prompt_template2.format_messages(description=description)

## Single Chain

In [ ]:
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain

llm = chat # ChatOpenAI(temperature=0.9, model=llm_model)

In [ ]:
llm = AzureChatOpenAI(temperature=0.0, model=deployment_name, azure_endpoint=azure_endpoint)

# previously template_string
description = '''The patient reports feeling unusually fatigued and thirsty over the past month, with increased urination, especially at night. The patient has also noticed unexplained weight loss despite eating regularly'''

classification_prompt = '''You are a medical expert. Based on the following patient symptoms and medical history,
    classify the most appropriate ICD-10 code from the descriptions provided.

    Patient Symptoms: {description}

    First classify the patient symptoms into one of following group:
    'A00–B99':'Certain infectious and parasitic diseases',
    'C00–D48':'Neoplasms',
    'D50–D89':'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism',
    'E00–E90':'Endocrine, nutritional and metabolic diseases',
    'F00–F99':'Mental and behavioural disorders',
    'G00–G99':'Diseases of the nervous system',
    'H00–H59':'Diseases of the eye and adnexa',
    'H60–H95':'Diseases of the ear and mastoid process',
    'I00–I99':'Diseases of the circulatory system',
    'J00–J99':'Diseases of the respiratory system',
    'K00–K93':'Diseases of the digestive system',
    'L00–L99':'Diseases of the skin and subcutaneous tissue',
    'M00–M99':'Diseases of the musculoskeletal system and connective tissue',
    'N00–N99':'Diseases of the genitourinary system',
    'O00–O99':'Pregnancy, childbirth and the puerperium',
    'P00–P96':'Certain conditions originating in the perinatal period',
    'Q00–Q99':'Congenital malformations, deformations and chromosomal abnormalities',
    'R00–R99':'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified',
    'S00–T98':'Injury, poisoning and certain other consequences of external causes',
    'V01–Y98':'External causes of morbidity and mortality',
    'Z00–Z99':'Factors influencing health status and contact with health services',
    'U00–U99':'Codes for special purposes'
    and then use this group to classify the most accurate ICD-10 code.
    Please provide the most relevant ICD-10 code and a brief explanation for the classification.

    Format the output as JSON with the following key it should be only group name eg. 'A00–B99', 'Z00–Z99':
    ICD code group
'''
prompt = ChatPromptTemplate.from_template(
   classification_prompt
)
chain = LLMChain(llm=llm, prompt=prompt)

ICD_group = chain.run(description)

Retrival_query = f"""
Find the most accurate ICD-10 code from the descriptions and the identified ICD group.

Description : {description}
ICD_group: {ICD_group}
"""

results = loaded_faiss.similarity_search_with_score(Retrival_query, k=5)

contexts = [result[0].page_content for result in results]

template_string = """
You are a medical expert. Based on the following patient symptoms and medical history,
classify the most accurate ICD-10 code from the description, the probable ICD group and additional context.

Description : {description}
ICD_group: {ICD_group}
Context: {context}

Format the output as JSON with the following keys:
ICD code
Explanation
"""

# prompt_template
prompt_template = ChatPromptTemplate.from_template(template_string)

prompt = prompt_template.format_messages(
                    description=description,
                    context=contexts,
                    ICD_group=ICD_group
                    )

ICD_Cls = chat(prompt)

print(ICD_Cls.content)

In [ ]:
# previously template_string
description = '''The patient reports feeling unusually fatigued and thirsty over the past month, with increased urination, especially at night. The patient has also noticed unexplained weight loss despite eating regularly'''

classification_prompt = '''You are a medical expert. Based on the following patient symptoms and medical history,
    classify the most appropriate ICD-10 code from the descriptions provided.

    Patient Symptoms: {description}

    First classify the patient symptoms into one of following group:
    'A00–B99':'Certain infectious and parasitic diseases',
    'C00–D48':'Neoplasms',
    'D50–D89':'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism',
    'E00–E90':'Endocrine, nutritional and metabolic diseases',
    'F00–F99':'Mental and behavioural disorders',
    'G00–G99':'Diseases of the nervous system',
    'H00–H59':'Diseases of the eye and adnexa',
    'H60–H95':'Diseases of the ear and mastoid process',
    'I00–I99':'Diseases of the circulatory system',
    'J00–J99':'Diseases of the respiratory system',
    'K00–K93':'Diseases of the digestive system',
    'L00–L99':'Diseases of the skin and subcutaneous tissue',
    'M00–M99':'Diseases of the musculoskeletal system and connective tissue',
    'N00–N99':'Diseases of the genitourinary system',
    'O00–O99':'Pregnancy, childbirth and the puerperium',
    'P00–P96':'Certain conditions originating in the perinatal period',
    'Q00–Q99':'Congenital malformations, deformations and chromosomal abnormalities',
    'R00–R99':'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified',
    'S00–T98':'Injury, poisoning and certain other consequences of external causes',
    'V01–Y98':'External causes of morbidity and mortality',
    'Z00–Z99':'Factors influencing health status and contact with health services',
    'U00–U99':'Codes for special purposes'
    and then use this group to classify the most accurate ICD-10 code.
    Please provide the most relevant ICD-10 code and a brief explanation for the classification.

    Format the output as JSON with the following key it should be only group name eg. 'A00–B99', 'Z00–Z99':
    ICD code group
'''
prompt = ChatPromptTemplate.from_template(
   classification_prompt
)
chain = LLMChain(llm=llm, prompt=prompt)

ICD_group = chain.run(description)

ICD_group

'"E00–E90"'

In [ ]:
Retrival_query = f"""
Find the most accurate ICD-10 code from the descriptions and the identified ICD group.

Description : {description}
ICD_group: {ICD_group}
"""

results = loaded_faiss.similarity_search_with_score(Retrival_query, k=5)

contexts = [result[0].page_content for result in results]
# print('*'*20,'Retrival_query', Retrival_query)

# print('*'*20,'contexts')
# for i, context in enumerate(contexts):
#   print('*'*10,f'context {i}')
#   print(context)

In [22]:
template_string = """
You are a medical expert. Based on the following patient symptoms and medical history,
classify the most accurate ICD-10 code from the description, the probable ICD group and additional context.

Description : {description}
ICD_group: {ICD_group}
Context: {context}

Format the output as JSON with the following keys:
ICD code
Explanation
"""

In [23]:
# prompt_template
prompt_template = ChatPromptTemplate.from_template(template_string)

prompt = prompt_template.format_messages(
                    description=description,
                    context=contexts,
                    ICD_group=ICD_group
                    )

ICD_Cls = chat(prompt)

print(ICD_Cls.content)

{
  "ICD code": "E10.65",
  "Explanation": "Type 1 diabetes mellitus with hyperglycemia"
}


In [ ]:
# prompt_template
prompt_template2 = ChatPromptTemplate.from_template(template_string)

prompt2 = prompt_template2.format_messages(
                    description=description,
                    ICD_group=ICD_group,
                    context=contexts)

chain2 = LLMChain(llm=llm, prompt=prompt2)


result = chain.run(description=description,
                    ICD_group=ICD_group,
                    context=contexts)

result

ValidationError: 1 validation error for LLMChain
prompt
  Input should be a valid dictionary or instance of BasePromptTemplate [type=model_type, input_value=[HumanMessage(content='\n..., response_metadata={})], input_type=list]
    For further information visit https://errors.pydantic.dev/2.9/v/model_type

# Chain with Rag

In [ ]:
from langchain.prompts import PromptTemplate
# from langchain.vectorstores import FAISS

# Assuming you have a FAISS index saved and ready for retrieval
faiss_index = FAISS.load_local(index_path, embeddings=embedding_model, allow_dangerous_deserialization=True)

In [ ]:
classification_prompt = '''You are a medical expert. Based on the following patient symptoms and medical history,
    classify the most appropriate ICD-10 code from the descriptions provided.

    Patient Symptoms: {description}

    First classify the patient symptoms into one of following group:
    'A00–B99':'Certain infectious and parasitic diseases',
    'C00–D48':'Neoplasms',
    'D50–D89':'Diseases of the blood and blood-forming organs and certain disorders involving the immune mechanism',
    'E00–E90':'Endocrine, nutritional and metabolic diseases',
    'F00–F99':'Mental and behavioural disorders',
    'G00–G99':'Diseases of the nervous system',
    'H00–H59':'Diseases of the eye and adnexa',
    'H60–H95':'Diseases of the ear and mastoid process',
    'I00–I99':'Diseases of the circulatory system',
    'J00–J99':'Diseases of the respiratory system',
    'K00–K93':'Diseases of the digestive system',
    'L00–L99':'Diseases of the skin and subcutaneous tissue',
    'M00–M99':'Diseases of the musculoskeletal system and connective tissue',
    'N00–N99':'Diseases of the genitourinary system',
    'O00–O99':'Pregnancy, childbirth and the puerperium',
    'P00–P96':'Certain conditions originating in the perinatal period',
    'Q00–Q99':'Congenital malformations, deformations and chromosomal abnormalities',
    'R00–R99':'Symptoms, signs and abnormal clinical and laboratory findings, not elsewhere classified',
    'S00–T98':'Injury, poisoning and certain other consequences of external causes',
    'V01–Y98':'External causes of morbidity and mortality',
    'Z00–Z99':'Factors influencing health status and contact with health services',
    'U00–U99':'Codes for special purposes'
    and then use this group to classify the most accurate ICD-10 code.
    Please provide the most relevant ICD-10 code and a brief explanation for the classification.

    Format the output as JSON with the following key it should be only group name eg. 'A00–B99', 'Z00–Z99':
    ICD code group
'''

# Create a Langchain prompt template for classification
classification_prompt_template = PromptTemplate(
    input_variables=["description"],
    template=classification_prompt
)

'"E00–E90"'

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [ ]:
# Define a refined classification prompt that takes context into account
refined_classification_prompt = """
You are a medical expert. Based on the following patient symptoms and medical history,
classify the most accurate ICD-10 code from the descriptions and the additional context provided.

Description: {description}
Context: {context}
"""
refined_classification_prompt_template = PromptTemplate(
    input_variables=["description", "context"],
    template=refined_classification_prompt
)

# Create an LLM chain with OpenAI model to generate refined classification
llm = chat
# llm = OpenAI(model="text-davinci-003", temperature=0)
refined_classification_chain = LLMChain(
    llm=llm,
    prompt=refined_classification_prompt_template
)
refined_classification_chain.run(description)

def retrieve_context(group: str, query: str):
    # Use the FAISS index to retrieve context based on the classified group
    # You could tweak the query based on group, or use a more specific query per group
    results = faiss_index.similarity_search(query, k=5)  # Top 3 results
    context = [result[0].page_content for result in results]

    return context

from langchain.chains import SimpleChain

def classify_description(description: str):
    # Step 1: Initial Classification
    initial_classification = classification_prompt_template.format(description=description)
    initial_group = llm(initial_classification)

    # Step 2: Retrieve Context from FAISS for the classified group
    context = retrieve_context(initial_group.strip(), description)

    # Step 3: Refined Classification
    refined_classification_input = refined_classification_prompt_template.format(
        description=description, context=context
    )
    refined_classification = llm(refined_classification_input)

    return refined_classification

# description = "A breakthrough in AI technology is changing the way businesses operate, with new AI models becoming more efficient."

# Run the chain
refined_category = classify_description(description)
print(refined_category)

## Sequential Chain

In [ ]:

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(template_string)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.run(product)

In [ ]:
# Function to classify a patient's description
def classify_patient_description(description):
    # Generate embedding for the patient description
    description_embedding = embedding_model.embed_query(description)

    # Perform a similarity search in the FAISS index
    results = faiss_index.similarity_search(description_embedding, k=5)  # Get top 5 matches
    return results

# Example patient description
patient_description = ("The patient reports feeling unusually fatigued and thirsty over the past month, "
                       "with increased urination, especially at night. The patient has also noticed "
                       "unexplained weight loss despite eating regularly. There is a family history of diabetes, "
                       "and the patient has not been physically active in recent years. No prior diagnosis of "
                       "diabetes, but blood sugar levels were borderline high during a previous checkup.")

# Classify the patient's description
classification_results = classify_patient_description(patient_description)

# Output the classification results
for result in classification_results:
    print(f"ICD Code: {result.metadata['icd_code']}, Description: {result.metadata['description']}")


In [ ]:
 # Or specify the full path where the .env file is stored

# # Now you can access the variables
# my_var = os.getenv('OPENAI_API_TYPE')  # Replace 'MY_VAR' with the name of the variable you want to access
# print(my_var)

True